Test och se om vi kan bättre utnnytja fält
* sbl_link --> Wikidata Property 3217 = Svenskt Biografiskt Lexikon
* skbl_link --> Wikidata Property 4963 = Svenskt kvinnobiografiskt lexikon
* sol_link ?!?!?!
* wikidata_id = Wikidata Qnummer
* wikipedia = WIkipedia URL

In [1]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)

Last run:  2021-10-01 11:46:50.645249


In [2]:
import urllib3, json
import pandas as pd 
http = urllib3.PoolManager() 
pd.set_option("display.max.columns", None) 
 
url = "https://litteraturbanken.se/api/get_authors"
r = http.request('GET', url)
data = json.loads(r.data)
df = pd.json_normalize(data["data"])

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4024 entries, 0 to 4023
Data columns (total 50 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   authorid                       4024 non-null   object 
 1   authorid_norm                  4024 non-null   object 
 2   db_checksum                    4024 non-null   object 
 3   db_timestamp                   4024 non-null   int64  
 4   doc_type                       4024 non-null   object 
 5   full_name                      4024 non-null   object 
 6   gender                         4024 non-null   object 
 7   imported                       3365 non-null   object 
 8   intro                          728 non-null    object 
 9   librisid                       2166 non-null   object 
 10  name_for_index                 4024 non-null   object 
 11  pictureinfo                    169 non-null    object 
 12  searchable                     4024 non-null   b

In [4]:
#just objects with show
dfShow = df[df["show"]].copy() 

In [5]:
dfexternal = dfShow[{"wikidata.sbl_link","wikidata.skbl_link","wikidata.wikidata_id","wikidata.sol_link"
,"wikidata.wikidata_id","wikidata.wikipedia"}]

In [6]:
dfexternal.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3130 entries, 1 to 4023
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   wikidata.sbl_link     765 non-null    object
 1   wikidata.skbl_link    152 non-null    object
 2   wikidata.wikipedia    1752 non-null   object
 3   wikidata.sol_link     118 non-null    object
 4   wikidata.wikidata_id  2134 non-null   object
dtypes: object(5)
memory usage: 146.7+ KB


In [7]:
end = datetime.now()
print("Ended: ", end) 
print('Time elapsed (hh:mm:ass.ms) {}'.format(datetime.now() - start_time))

Ended:  2021-10-01 11:46:53.039808
Time elapsed (hh:mm:ass.ms) 0:00:02.395002
